In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import rasterio
import numpy as np
import torch.nn as nn
import torch.optim as optim
from skimage.transform import resize
from tqdm import tqdm
import pandas as pd
import torchvision.transforms as transforms

In [5]:
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in tqdm(dataloader):
        inputs  = inputs.float().to(device)
        targets = targets.float().to(device)

        inputs[torch.isnan(inputs)] = 0
        targets[torch.isnan(targets)] = 0

        assert not torch.isnan(inputs).any(), "Input contains NaN values"
        assert not torch.isnan(targets).any(), "Target contains NaN values"
        
        optimizer.zero_grad()
        outputs_up = model(inputs)

        loss = criterion(outputs_up, targets).to(device)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

def validate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader):
            inputs = inputs.float().to(device)
            targets =  targets.float().to(device)
            inputs[torch.isnan(inputs)] = 0
            targets[torch.isnan(targets)] = 0

            outputs = model(inputs)

            loss = criterion(outputs, targets)
            running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CALayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(CALayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv_du = nn.Sequential(
            nn.Conv2d(channel, channel // reduction, 1, padding=0, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(channel // reduction, channel, 1, padding=0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.conv_du(y)
        return x * y

class LAM_Module(nn.Module):
    def __init__(self, in_dim):
        super(LAM_Module, self).__init__()
        self.chanel_in = in_dim
        self.gamma = nn.Parameter(torch.zeros(1))
        self.softmax  = nn.Softmax(dim=-1)

    def forward(self,x):
        m_batchsize, C, height, width = x.size()
        proj_query = x.view(m_batchsize, C, -1)
        proj_key = x.view(m_batchsize, C, -1).permute(0, 2, 1)
        energy = torch.bmm(proj_query, proj_key)
        energy_new = torch.max(energy, -1, keepdim=True)[0].expand_as(energy)-energy
        attention = self.softmax(energy_new)
        proj_value = x.view(m_batchsize, C, -1)

        out = torch.bmm(attention, proj_value)
        out = out.view(m_batchsize, C, height, width)

        out = self.gamma*out + x
        return out

class CSAM_Module(nn.Module):
    def __init__(self, in_dim):
        super(CSAM_Module, self).__init__()
        self.chanel_in = in_dim
        self.conv = nn.Conv3d(1, 1, 3, 1, 1)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self,x):
        m_batchsize, C, height, width = x.size()
        out = x.unsqueeze(1)
        out = self.conv(out)
        out = out.squeeze(1)
        
        out = self.gamma*out + x
        return out

class RCAB(nn.Module):
    def __init__(self, n_feat, kernel_size=3, reduction=16):
        super(RCAB, self).__init__()
        modules_body = []
        modules_body.append(nn.Conv2d(n_feat, n_feat, kernel_size, padding=kernel_size//2))
        modules_body.append(nn.ReLU(inplace=True))
        modules_body.append(nn.Conv2d(n_feat, n_feat, kernel_size, padding=kernel_size//2))
        modules_body.append(CALayer(n_feat, reduction))
        self.body = nn.Sequential(*modules_body)

    def forward(self, x):
        res = self.body(x)
        res += x
        return res

class HAN(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, n_feats=64, n_resgroups=10, n_resblocks=20, reduction=16, scale=10):
        super(HAN, self).__init__()
        
        self.conv_first = nn.Conv2d(in_channels, n_feats, kernel_size=3, padding=1)
        
        modules_body = []
        for _ in range(n_resgroups):
            modules_body.append(
                nn.Sequential(*[RCAB(n_feats, reduction=reduction) for _ in range(n_resblocks)])
            )
        
        self.body = nn.Sequential(*modules_body)
        
        self.lam = LAM_Module(n_feats)
        self.csam = CSAM_Module(n_feats)
        
        self.conv_last = nn.Conv2d(n_feats, n_feats, kernel_size=3, padding=1)
        
        self.upsampler = nn.Sequential(
            nn.Conv2d(n_feats, n_feats * 25, kernel_size=3, padding=1),
            nn.PixelShuffle(5),
            nn.Conv2d(n_feats, n_feats * 4, kernel_size=3, padding=1),
            nn.PixelShuffle(2),
            nn.Conv2d(n_feats, out_channels, kernel_size=3, padding=1)
        )

    def forward(self, x):
        x = self.conv_first(x)
        res = x
        
        for module in self.body:
            res = module(res)
        
        res = self.lam(res)
        res = self.csam(res)
        res = self.conv_last(res)
        
        res += x
        
        x = self.upsampler(res)
        return x

Input shape: torch.Size([1, 1, 40, 52])
Output shape: torch.Size([1, 1, 400, 520])


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = HAN(in_channels=1, out_channels=1, n_feats=64, n_resgroups=10, n_resblocks=20, reduction=16, scale=10).to(device)

learning_rate = 0.0001
criterion = nn.L1Loss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)

num_epochs = 100
best_loss = float('inf')

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Model is training on: {next(model.parameters()).device}')
    train_loss = train_model(model, dataloader_train, criterion, optimizer, device)
    val_loss = validate_model(model, dataloader_val, criterion, device)
    print(f'Train Loss: {train_loss:.4f} | Validation Loss: {val_loss:.4f}')
    
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), 'path')
        print('Model saved!')


cuda
Epoch 1/100
Model is training on: cuda:0


100%|██████████| 229/229 [00:44<00:00,  5.12it/s]


Train Loss: 1422.8498 | Validation Loss: 33.1547
Model saved!
Epoch 2/100
Model is training on: cuda:0


100%|██████████| 229/229 [00:44<00:00,  5.17it/s]


Train Loss: 27.9682 | Validation Loss: 20.9942
Model saved!
Epoch 3/100
Model is training on: cuda:0


...

Train Loss: 1.1952 | Validation Loss: 0.7018
Model saved!
Epoch 99/100
Model is training on: cuda:0


100%|██████████| 229/229 [00:44<00:00,  5.16it/s]


Train Loss: 0.6574 | Validation Loss: 0.5786
Model saved!
Epoch 100/100
Model is training on: cuda:0


100%|██████████| 229/229 [00:44<00:00,  5.17it/s]

Train Loss: 1.2893 | Validation Loss: 0.9916
